## Module submission header
### Submission preparation instructions 
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Xi Chen
    - Email: xc98@drexel.edu
- Group member 2
    - Name: Tai Nguyen
    - Email: tdn47@drexel.edu
- Group member 3
    - Name: Tien Nguyen
    - Email: thn44@drexel.edu
- Group member 4
    - Name: Raymond Yung
    - Email: raymond.yung@drexel.edu

### Additional submission comments
- Tutoring support received: NA
- Other (other): NA

# DSCI 691: Natural language processing with deep learning <br> Assignment 2: Abstracting Summaries of the News
## Data and Utilities 
Here, we'll be working again with the same linked NewsTweet data and some essential utilities presented in the __Chapter 1 Notes__.

In [67]:
import json
newstweet = json.load(open('./data/newstweet-subsample-linked.json'))
# exec(open('./01-utilities.py').read())
from utilities import *

## Overview 
The purpose of this assignment (37 pts) is to gain some experience with the extremely important NLP task called language modeling (LM'ing). We'll explore a traditional $n$-gram approach to help elucidate the central challenges with the problem.

Since this is an LM'ing assignment, for the sanity checks we'll be working on a single document from the data set throughout, focused on a Robert Downey Jr. movie. But in principle you should be able to apply this assignment to any of the articles and generate text for summaries, and you should&mdash;it's fun!

Here's the article of focus:

In [68]:
print(newstweet[5]['text'])

Robert Downey Jr. would rather talk to animals than people as the new Dr. Dolittle. But, as seen this official “Dolittle” trailer released Sunday, he is soon forced to leave his hideaway and set sail across the sea to a mythical island.

“We have no choice but to embark on this perilous journey,” Downey whispers to his feathered and furry friends.

And that involves getting into all sorts of dangerous situations, like being chained in a medieval dungeon with a tiger who greets him with, “hello, lunch” and then jumps him. But not to worry — a gorilla voiced by Rami Malek comes to his rescue.

Also Read: Universal Moves Robert Downey Jr.'s 'Voyage of Doctor Dolittle' to January 2020

“Dolittle” tells the story of famed doctor and veterinarian during the time of Queen Victoria’s England, Dr. John Dolittle, who returns to action after the loss of his wife seven years earlier caused him to become a hermit who only talks to animals. But when the young queen (Jessie Buckley, “Wild Rose”) fall

As we continue, we'll explore ways that we can use sparse models to regenerate this document, and along the way we'll get some experience with model-sampling and perplexity as a performance measure.

## Experiment

### 1. (3 pts) Build an $n$-gram counter
Given an input list of `tokens`, use list slices to complete the `count(tokens, n = 1)` function to produce and return the `ngram_counts` object, as a `Counter()` of `n`-sized tuples.

In [69]:
# A1:Function(3/3)

def count(tokens, n = 1):
    
    #--- your code starts here
    ngram_counts = Counter([tuple(tokens[i: i+n]) for i in range(len(tokens)-n +1)])
    
    #--- your code stops here
    
    return ngram_counts


In [70]:
# A1:SanityCheck

count(["this", " ", "is", " ", "an", " ", "example", " ", 
       "of", " ", "a", " ", "token", " ", "stream"], 5)

Counter({('this', ' ', 'is', ' ', 'an'): 1,
         (' ', 'is', ' ', 'an', ' '): 1,
         ('is', ' ', 'an', ' ', 'example'): 1,
         (' ', 'an', ' ', 'example', ' '): 1,
         ('an', ' ', 'example', ' ', 'of'): 1,
         (' ', 'example', ' ', 'of', ' '): 1,
         ('example', ' ', 'of', ' ', 'a'): 1,
         (' ', 'of', ' ', 'a', ' '): 1,
         ('of', ' ', 'a', ' ', 'token'): 1,
         (' ', 'a', ' ', 'token', ' '): 1,
         ('a', ' ', 'token', ' ', 'stream'): 1})

For reference, your output should be:

```
Counter({(' ', 'a', ' ', 'token', ' '): 1,
         (' ', 'an', ' ', 'example', ' '): 1,
         (' ', 'example', ' ', 'of', ' '): 1,
         (' ', 'is', ' ', 'an', ' '): 1,
         (' ', 'of', ' ', 'a', ' '): 1,
         ('a', ' ', 'token', ' ', 'stream'): 1,
         ('an', ' ', 'example', ' ', 'of'): 1,
         ('example', ' ', 'of', ' ', 'a'): 1,
         ('is', ' ', 'an', ' ', 'example'): 1,
         ('of', ' ', 'a', ' ', 'token'): 1,
         ('this', ' ', 'is', ' ', 'an'): 1})
```

## 2. (4 pts) Build $n$-gram frequencies up to a size
Here, your job will be to apply the `count` function to build $n$-gram frequency distributions up to a given maximum size. To do this, complete the `make_ngram_frequency(documents, n = 1, space = True)`
The function's main argument is `documents`, which will be a list of strings, and the function's only output will be `ngram_frequencies`, which will be a list of $n$-gram `Counter()`s, up to a specified (by `n`) size.

In [71]:
# A2:Function(4/4)

def make_ngram_frequency(documents, n = 1, space = True):
    ngram_frequencies = []
    
    #--- your code starts here
    for i in range(1, n+1):
        counter_n = Counter()
        for doc in documents:
            counter_n.update(count(tokenize(doc, space=space), n=i))
        ngram_frequencies.append(counter_n)
    #--- your code stops here
    return ngram_frequencies, {t[0]: i for i, t in enumerate(ngram_frequencies[0].keys())}


In [72]:
# A2:SanityCheck

n = 9
ngram_frequencies, type_index = make_ngram_frequency([x['text'].lower() for x in newstweet], n = n)
ngram_frequencies[5][tuple(tokenize('robert downey jr.'))]

8

For reference, your output should be:
```
8
```

## 3. (6 pts) Build the standard LM
Now it's time for the model, i.e., computing the LM probabilities from __Section 2.1.4.1__. This will entail completing the two tricks, the first of which, $\varepsilon$-smoothing we re-define (for stability) for some small $\varepsilon>0$ as:
$$
\hat{P}(t_n|t_1, t_2, \cdots t_{n-1}) = \frac{\frac{\varepsilon}{|W|} + f(t_1, t_2,\cdots, t_n)}{\varepsilon + f(t_1, t_2,\cdots, t_{n-1})}
$$
where some small, constant non-zero weight ($\varepsilon$) is distributed to each type of the model in _every_ context, regardless of it's actual appearance in the data.

The second component you'll have to sort out is _backoff_, where the desired probabilities are approximated via the next-lower-$n$ context adjacent to the prediction point:
$$
\hat{P}(t_n|t_1, \cdots t_{n-1})\approx\hat{P}(t_n|t_2, \cdots t_{n-1}).
$$

For both cases, this amounts to determining `t_Ps`, as a `Counter()` of types-as-keys with probability values and thus completing the function:
```
P_next(gram, ngram_frequencies, type_index, epsilon = 0.1)
```
for which `gram` corresponds to the vector, $\vec{t} = [t_1,\cdots,t_{n-1}]$ of tokens preceeding the prediction point.  

In this, $\varepsilon_w$  (coded as `epsw`) will indicate _the portion of the total mass of the smoothing parameter, $\varepsilon$, assigned to $w$_. Hence, the default setting `epsilon = 0.1` will mean:
$$
\varepsilon_w = \frac{0.1}{|W|}.
$$
which should allow for convienient parameterization of _slight_ smoothings, which won't totally swamp the model's performance.

[Hint. use the `n1` (context size) to navigate the `ngram_frequencies` object, and don't be afraid to slice `gram`s].

In [73]:
# A3:Function(6/6)

def P_next(gram, ngram_frequencies, type_index, epsilon = .1):
    n1 = len(gram)
    epsw = epsilon/len(type_index)
    if gram in ngram_frequencies[n1-1]: ## use gram to condition frequencies with epsilon-smoothing
        
        #--- your code starts here
        prob_gram = ngram_frequencies[n1-1][gram] + epsilon
        t_Ps = {}
        for tok in type_index:
            next_gram = tuple(list(gram) + [tok])
            if next_gram in ngram_frequencies[n1]:
                prob_next_tok = ngram_frequencies[n1][next_gram] + epsw
            else:
                prob_next_tok = epsw
            t_Ps[tok] = prob_next_tok / prob_gram
            
        t_Ps = Counter(t_Ps)
        #--- your code stops here
        
    else: ## recursively back off to lower-n model
        
        #--- your code starts here
        t_Ps = P_next(gram[1:], ngram_frequencies, type_index, epsilon)
        #--- your code stops here

    
    return t_Ps

For reference, your output should be:
```
[1.0,
 [('adventure', 0.9090937517766786),
  ('aew', 2.8426857695150374e-06),
  (' ', 2.8426857695150374e-06),
  ('star', 2.8426857695150374e-06),
  ('matt', 2.8426857695150374e-06),
  ('jackson', 2.8426857695150374e-06),
  ('of', 2.8426857695150374e-06),
  ('the', 2.8426857695150374e-06),
  ('young', 2.8426857695150374e-06),
  ('bucks', 2.8426857695150374e-06)],
 [(' ', 0.9090937517766786),
  ('aew', 2.8426857695150374e-06),
  ('star', 2.8426857695150374e-06),
  ('matt', 2.8426857695150374e-06),
  ('jackson', 2.8426857695150374e-06),
  ('of', 2.8426857695150374e-06),
  ('the', 2.8426857695150374e-06),
  ('young', 2.8426857695150374e-06),
  ('bucks', 2.8426857695150374e-06),
  ('is', 2.8426857695150374e-06)]]
```

In [74]:
# A3:SanityCheck

[np.nansum([x for x in P_next(tuple(tokenize("he goes on an ")), ngram_frequencies, type_index).values()]), 
 list(P_next(tuple(tokenize("he goes on an ")), ngram_frequencies, type_index).most_common(10)), 
 list(P_next(tuple(tokenize(" he goes on an")), ngram_frequencies, type_index).most_common(10))]

[1.0,
 [('adventure', 0.9090937517766786),
  ('aew', 2.8426857695150374e-06),
  (' ', 2.8426857695150374e-06),
  ('star', 2.8426857695150374e-06),
  ('matt', 2.8426857695150374e-06),
  ('jackson', 2.8426857695150374e-06),
  ('of', 2.8426857695150374e-06),
  ('the', 2.8426857695150374e-06),
  ('young', 2.8426857695150374e-06),
  ('bucks', 2.8426857695150374e-06)],
 [(' ', 0.9090937517766786),
  ('aew', 2.8426857695150374e-06),
  ('star', 2.8426857695150374e-06),
  ('matt', 2.8426857695150374e-06),
  ('jackson', 2.8426857695150374e-06),
  ('of', 2.8426857695150374e-06),
  ('the', 2.8426857695150374e-06),
  ('young', 2.8426857695150374e-06),
  ('bucks', 2.8426857695150374e-06),
  ('is', 2.8426857695150374e-06)]]

### 4. (7 pts) Build a model sampler
Now that we have a LM, we need a way to sample from it. To start complete the function:
```
sample_LM(gram, LM_args, top = 1., LM = P_next)
``` 
which must perform a weghted random sample via `np.random.choice()`, using the `gram` for the context of a prediction point (as in __Part 4.__, for `P_next()`). However, this sampler must deploy one of two sampling algorithms, as specified by the `top` parameter. Specifically:
1. when `type(top) == float`, the floating point value of `top` should represent the cumulative probabiliy of top-scoring predictions to weight a sample from; and
2. when `type(top) == int`, the integer value of `top` should represent the `top` highest-ranking predicitons to weight a sample from.

In case (1), the sample might range over many or few possibilities, depending on the confusion of the model at the point of the prediction, and in case (2) the sample might be constrained to a limited vocabulary at each step. However, in both your code should use, e.g., a boolean mask to filter the `Ps` (prediciton probabilities) and `ts` (prediction types) down to just those in the `top`, i.e., 'viable' set that will be passed to the sampler.

Note: in both cases your filtered prediction probabilities (`Ps`) must be re-normalized for the weighted random sample!

In [75]:
# A4:Function(7/7)

def sample_LM(gram, LM_args, top = 1., LM = P_next):
    
    Ps = LM(gram, *LM_args)
    ts, Ps = map(np.squeeze, zip(*Ps.most_common()))
    Ps /= Ps.sum()
    
    #--- your code starts here
    if isinstance(top, float):
        indices = (top - Ps.cumsum()) >= 0
        Ps = Ps[indices]
        ts = ts[indices]
    elif isinstance(top, int):
        Ps = Ps[:top]
        ts = ts[:top]

    Ps /= Ps.sum()
        
    #--- your code stops here
    
    s = np.random.choice(ts, size=1, replace=False, p=Ps)[0]
    
    return s
    

For reference, your output should be:
```
'adventure'
```

In [76]:
# A4:SanityCheck

np.random.seed(691)
sample_LM(tuple(tokenize("he goes on an ")), (ngram_frequencies, type_index, 0.01))

'adventure'

### 5. (5 pts) Build a recitation function for the LM
Here, our goal will be to have the LM 'recite' a given `document` string input. In particular, your job is to complete the function:
```
recitation, likelihood = recite(document, LM_args, LM = P_next, n = 5, top = 1., verbose = True)
```
which has the following arguments:
- `document`: a string to be modeled by its ngrams
- `LM_args`: a tuple of all arguments to be passed to the LM
- `LM (= P_next)`: the LM function to be operated for the recitation
- `n (= 5)`: an integer number indicating the gram-size to model from
- `top (= 1.)`: the sampling paramater for the `sample_LM` function
- `verbose (= True)`: a boolean, indicating whether the model should print the text it produces, while operating

and has the following return values are:
- `recitation`: a list of the tokens which the LM _predicts_, in order
- `likelihood`: a list of the probabilities for the _correct_ targets of the LM as it operates

In [77]:
# A5:Function(5/5)

def recite(document, LM_args, LM = P_next, n = 5, top = 1., verbose = True):
    tokens = tokenize(document)
    ngram_stream = [tuple(tokens[i:i+n]) for i in range(0,len(tokens) - n + 1)]

    if verbose:
        print("generated document, starting from \""+"".join(ngram_stream[0][:-1])+"\":\n")
        
    likelihood = []; recitation = []
    for ix, ngram in enumerate(ngram_stream):
        
        #--- your code starts here
        target = ngram[-1]
        n1gram = tuple(ngram[:-1])
        Ps = LM(n1gram, *LM_args)
        likelihood.append(Ps[target])
        recitation.append(sample_LM(n1gram, LM_args, LM = LM, top = top))
        
        #--- your code stops here
        
        if verbose:
            print(recitation[-1], end = '')
    
    return recitation, likelihood

For reference, your output should be:
```
generated document, starting from "robert downey ":

jr. ( aikman have to patients than people as of earth batman. birx  but, for i this?official portdolittle” is released sunday, either wasn't not forced to “ nikola post and set them across the country,( a bout island.

athe need to higher but to get on quests backdrop journey,” downey whispers to his hamstring and furry friends.

",this’your lots into all of of dangerous situations, like me chained in a federal dungeon with a similar who greets him with  buthello, lunch” and it they him. he even penetrating receive about a businessman voiced by none malek comes to takedown fans.

also,overshadowed: delta moves robert downey jr.  'voyage of doctor credentialing to january 2020 
floyddolittle” trailer the moderator a the doctor and whistleblower during the early these year victoria’s england, dismissed. myron dolittle, who are to service after the police,of his war dorothy years,of,caused him to be the thing who only talks to appear  but there i names,hawaiian (jessieconsiderationbuckley, “wild rose”) falls ill, he has down sale industrial to find out a would-be. remember played choosing on stage usa by a bipartisan voter (harry collett, “dunkirk”) and the individuals friends, located at anxious gorilla (malek), an organization duck (octavia spencer), an atlanta-based ostrich (kumail nanjiani), an abolitionist polar bear (john cena), and if chest-strapped parrot (emma thompson).

the brown's also stars as the, along with added sheen as mudfly. additional reporting performers include marion cotillard, frances de la marihuana, 1972 ejogo, ralph fiennes, selena was  27 jurich, and other robinson.

“he” is a more james hawking (“syriana,” “traffic”), as the by nadal roth and jeff kirschenbaum under their “/kirschenbaum films banner (“alice in wonderland,” “calf”), and of as the susan downey,(“sherlock holmes” franchise, aewthe power”) for $ downey. downey jr. do produces along iwth sarah bradshaw (“the mummy,” “traffic”),and quarterback roth (“maleficent: mistress of evil ringed.
```

In [78]:
# A5:SanityCheck

j = 5
document = newstweet[j]['text'].lower()
np.random.seed(691)
recitation, likelihood = recite(document, (ngram_frequencies, type_index, 0.01))

generated document, starting from "robert downey ":

jr. ( aikman have to patients than people as of earth batman. birx  but, for i this?official portdolittle” is released sunday, either wasn't not forced to “ nikola post and set them across the country,( a bout island.

athe need to higher but to get on quests backdrop journey,” downey whispers to his hamstring and furry friends.

",this’your lots into all of of dangerous situations, like me chained in a federal dungeon with a similar who greets him with  buthello, lunch” and it they him. he even penetrating receive about a businessman voiced by none malek comes to takedown fans.

also,overshadowed: delta moves robert downey jr.  'voyage of doctor credentialing to january 2020 
floyddolittle” trailer the moderator a the doctor and whistleblower during the early these year victoria’s england, dismissed. myron dolittle, who are to service after the police,of his war dorothy years,of,caused him to be the thing who only talks to appear  b

### 6. (2 pts) Build a perplexity performance evaluator
We want to know how well this LM works, so let's compute the _average_ perplexity with respect to the document's stream of tokens (prediction points): $t_1, \cdots, t_m$. For each $i=1,\cdots,m$ of these, let $\hat{y}_i\in[0,1]^{|W|}$ be the probabilistic prediction vector over the vocabulary, so that $\hat{y}_{i,t_i}$ is the prediction probability for the _correct_ type, $t_i$ at the $i^\text{th}$ prediction point. Under this notation, we wish to compute the perplexity across our a document:
$$
\mathcal{T}(t_1,\cdots,t_m) = e^{
    -\frac{1}{m}\sum_{i = 1}^m\log{\hat{y}_{i,t_i}}
}
$$
In order to do this, we'll have to work from the `recite()` function's `likelihood` output format, which should now be a `list` of the $\hat{y}_i\in[0,1]^{|W|}$ values. 

With this all in mind, your job is to complete the `perplexity(likelihood)`, which returns a floating point number named `average_perplexity` (computed as above).


In [79]:
# A6:Function(2/2)

def perplexity(likelihood):
    
    #--- your code starts here
    norm_coeff = 1/len(likelihood)
    average_perplexity = np.exp(-norm_coeff*np.log(likelihood).sum())
    #--- your code stops here
    
    return average_perplexity

For reference, your output should be:
```
average perplexity of recitation:  1.8009612714103027
```

In [80]:
# A6:SanityCheck

print("average perplexity of recitation: ", perplexity(likelihood))

average perplexity of recitation:  1.8009546437644577


### 7. (6 pts) Build a rambling function for the LM
Here, our goal will be to have the LM 'ramble' from a given `prompt` of token-stream (list of strings) input. Note: while this function can go 'off the script', it must start from a prompt within its vocabulary, as specified within `LM_args[-1]`, i.e., `type_index` object. In particular, you must complete the function:
```
rambling, likelihood = ramble(prompt, docsize, LM_args, LM = P_next, n = 5, top = 1., verbose = True)
```
which accepts the following arguments:
- `prompt`: a list of strings (tokens) which define the starting ngram for rambling prediction
- `docsize`: the integer number of tokens to generate in the `rambling`
- `LM_args`: same as for `recite()`
- `LM (= P_next)`: same as for `recite()`
- `n (= 5)`: same as for `recite()`
- `top (= 1.)`: same as for `recite()`
- `verbose (= True)`: same as for `recite()`

and has the following return values are:
- `rambling`: like `recitation`, a list of the tokens which the LM _predicts_, in order
- `likelihood`: _now_, a list of the probabilities for the _predictions_ made by the LM as it operates

In [81]:
# A7:Function(6/6)

def ramble(prompt, docsize, LM_args, LM = P_next, n = 5, top = 1., verbose = True):

    if verbose:
        print("generated document, starting from \""+"".join(prompt)+"\":\n")
    
    likelihood = []; rambling = []
    n1gram = prompt[-n:]
    while len(rambling) < docsize:
        
        #--- your code starts here
        Ps = LM(n1gram, *LM_args)
        next_tok = sample_LM(n1gram, LM_args, top, LM)
        likelihood.append(Ps[next_tok])
        rambling.append(next_tok)
        n1gram = tuple(list(n1gram[-n+1:]) + [next_tok])

        #--- your code stops here
        
        if verbose:
            print(rambling[-1], end = '')
    return rambling, likelihood

For reference, your output should be:
```
generated document, starting from "robert downey jr":

. do with this issue, might also like:

of course, not everything that excited the beautifully glowing phosphors of a high 

average perplexity of ramble:  1.6209028459433603
```

In [82]:
# A7:SanityCheck

j = 5
np.random.seed(691)
document = tokenize(newstweet[j]['text'].lower())
rambling, likelihood = ramble(tuple(document[:5]), 46, 
                              (ngram_frequencies, type_index, 0.01))
print("\n\naverage perplexity of ramble: ", perplexity(likelihood))

generated document, starting from "robert downey jr":

. do with this issue, might also like:

of course, not everything that excited the beautifully glowing phosphors of a high 

average perplexity of ramble:  1.6209028459433603


### 8. (4 pts) Constraining the vocabulary of a ramble
Since we'd like to summarize these news articles, an easy trick to get the LM talk about the 'right stuff' is simply to constrain to the vocabulary of a given document. As such, we can and will make `type_index`-like objects for each article and then just use the same architecture as above.

So here, you must complete the `make_doc_types()`, which accepts a list of strings named `documents`, the overall `type_index`, and a usual `space` boolean parameter. This amounts to constructing the `doc_types` object as a list of dictionaries, each of which has the same format as `type_index`, with the caveat, that each of `doc_types[j]` should only contain the type-index mapping for its given `j`th `document`, from `documents`.



In [83]:
# A8:Function(4/4)

def make_doc_types(documents, type_index, space = True):
    
    doc_types = []
    #--- your code starts here
    for doc in documents:
        _, doc_unique_toks = make_ngram_frequency([doc])
        doc_types.append({t: type_index[t] for t in doc_unique_toks})
        
    #--- your code stops here
        
    return doc_types

For reference, your output should be:
```
generated document, starting from "robert downey jr":

. and his wife and her friends as no people after the new england is nanjiani), an cynical ostrich (kumail nanjiani), an enthusiastic duck (octavia spencer), an enthusiastic duck (octavia spencer), an enthusiastic duck (octavia spencer), an upbeat polar bear (john cena), and all of the people that have the no. 

average perplexity of ramble:  2.26301542517039
```

In [84]:
# A8:SanityCheck

j = 5
np.random.seed(691)
document = tokenize(newstweet[j]['text'].lower())
doc_types = make_doc_types([x['text'].lower() for x in newstweet], type_index)
rambling, likelihood = ramble(tuple(document[:5]), 120, 
                              (ngram_frequencies, doc_types[j], 0.01))
print("\n\naverage perplexity of ramble: ", perplexity(likelihood))

generated document, starting from "robert downey jr":

. and his wife and her friends as no people after the new england is nanjiani), an cynical ostrich (kumail nanjiani), an enthusiastic duck (octavia spencer), an enthusiastic duck (octavia spencer), an enthusiastic duck (octavia spencer), an upbeat polar bear (john cena), and all of the people that have the no. 

average perplexity of ramble:  2.26301542517039
